# INFO 2950 Project Phase 2
#### Talia Boehm, Lucy Barsanti, Emma Shen, Maddi Klancher


Link to dataset: https://www.kaggle.com/datasets/bigquery/chrome-user-experience-report

In [7]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import seaborn
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'pandas'

## I. Research Question

### A. Instructions from student handbook 
State your research question (s) clearly.


## II. Data Collection and Cleaning
### A. Instructions from student handbook 

- Have an initial dra of your data cleaning
appendix. 
- Document every step that takes your raw data file(s) and turns it into the analysis-ready data set that you would submit with your final project. 
- Include text cells describing your data collection (downloading, Rescraping, surveys, etc), and text cells describing any additional data curation/cleaning (merging data frames, filtering, transformations of
variables, etc). 
- Include code cells for data curation/cleaning, but not
collection. 
- *Note:* You should be saving data in intermediate files at several points through this process so that you are not starting from scratch every time you change something. This also makes sure you have documentation for everything that you have done for your reference, but we do not necessarily need to see all of it in code form.

### Our work here

We are using the *Chrome User Experience Report* dataset that we found on Kaggle. 

Link to dataset: https://www.kaggle.com/datasets/bigquery/chrome-user-experience-report"


In [ ]:
ux_url = "https://www.kaggle.com/datasets/bigquery/chrome-user-experience-report"

ux_result = requests.get(ux_url)
if ux_result.status_code != 200:
    print("something went wrong:", ux_result.status_code, ux_result.reason)

with open("ux_report.html", "w") as writer:
    writer.write(ux_result.text)

with open("ux_report.html", "r") as reader:
    html_source = reader.read()

## III. Data Description

### A. Instructions from student handbook
Have an initial dra of your data description section.
Your data description should be about your analysis-ready data.


## IV. Data Limitation
### A. Instructions from student handbook 
Have an initial dra of your data description section.

Identify any potential problems with your dataset.

## V. Exploratory Data Analysis 
### A. Instructions from student handbook
Have an initial dra of your data description section.
Perform and initial exploratory data analysis 

## VI. Questions for reviewers 

### A. Instructions from student handbook
Have an initial dra of your data description section.
Perform and initial exploratory data analysis 
List specific questions for your peer reviewers
and project mentor to answer in giving you feedback on this phase.
